# Preview Quiz prompt

In [1]:
from pydantic import BaseModel
from openai import OpenAI
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [2]:
from jinja2 import Template

prompt_template = Template("""
[PURPOSE]
Preview lesson content through quizzes to give a sneak peek before the actual lesson.
The quizzes must ask key expressions/the most structurally important words themselves written in or based on the GOAL/DESCRIPTION.

[QUIZ GUIDELINES]
- Create two preview quizzes per topic. One from KEY_EXPRESSION, the other from SUB_EXPRESSIONS.
- Put blanks on the KEY_EXPRESSION and SUB_EXPRESSION itself, no elsewhere.
- No blanks or tildes in Korean translation
- No tildes in the quiz.
- Make sure you have NO multiple "equally valid" answers
- Blanks must directly ask the key expressions/the most meaningful words themselves based on the DESCRIPTION.
- Answer index = the position number of the answer.
- Ensure that the quiz is a verb, adjective, or a key phrase from the lesson.
- No proper nouns can be tested.
- Create SAMPLE SCRIPT for each topic at the end of each topic.


[POLICY]
Each quiz needs:
- A blank (test key patterns itself)
- Complete Korean translation (no blanks or tildes)
- Three options (one correct, two wrong)
- No multiple "equally valid" answers/options
- Wrong options must be distinctly wrong.
- Shuffle the order of answer choices='answer indexes (1-3)' to avoid predictability.
- SAMPLE SCRIPT must not be the same as the quizzes. Change one or two words to create a dialogue of different topic, but maintain the structure.

[Example]
```
Input: 
    {
    "DESCRIPTION": "This is my friend, ~와 같은 표현을 사용해 사람을 소개하기.",
    "KEY_EXPRESSION": "A: This is my friend, Jake.",
    "SUB_EXPRESSION": "A: Let me introduce my friend, ~. A: Meet my friend, ~."
  }

Output:
    {
    "goal": [
      "다른 사람을 소개하는 표현을 사용할 수 있어요.",
      "This is my friend, ~과 같은 표현을 사용하여 친구를 소개할 수 있어요."
    ],
    "preview_quizzes": [
      {
        "fill_in_blank_quiz": "___ is my friend, Jennie.",
        "korean_translation": "이쪽은 내 친구 Jennie야.",
        "options": ["I", "They", "This"],
        "answer": "This",
        "answer_index": 3
      },
      {
        "fill_in_blank_quiz": "Let me ___ my friend, Sarah.",
        "korean_translation": "내 친구 Sarah를 소개할게.",
        "options": ["introduce", "inter", "intro"],
        "answer": "introduce",
        "answer_index": 2
      }
    ]
    "sample_script" : [
      {
        "sample_script_eng" : "A: This is my friend, Emma. B: Nice to meet you, Emma! A: Let me introduce my friend, David. B: Hey, David! Glad to meet you.",
        "sample_script_kor" : "A: 얘는 내 친구, 엠마야! B: 반가워, 엠마! A: 내 친구 데이빗을 소개할게. B: 안녕, 데이빗! 만나서 반가워."
      }                       
    ]
  }
}
```

[Input]
```
DESCRIPTION: {{description}}
KEY EXPRESSION: {{key_expression}}
SUB EXPRESSION: {{sub_expression}}
```

""")

In [28]:
prompt = prompt_template.render(
    description = "should~ 를 활용해서 의무 표현하기",
    key_expression = "You should drink more water.",
    sub_expression = "You should apologize to her. You should follow the safety rules."
)

In [5]:
print(prompt)


[PURPOSE]
Preview lesson content through quizzes to give a sneak peek before the actual lesson.
The quizzes must ask key expressions/the most structurally important words themselves written in or based on the GOAL/DESCRIPTION.

[QUIZ GUIDELINES]
- Create two preview quizzes per topic. One from KEY_EXPRESSION, the other from SUB_EXPRESSIONS.
- Put blanks on the KEY_EXPRESSION and SUB_EXPRESSION itself, no elsewhere.
- No blanks or tildes in Korean translation
- No tildes in the quiz.
- Make sure you have NO multiple "equally valid" answers
- Blanks must directly ask the key expressions/the most meaningful words themselves based on the DESCRIPTION.
- Answer index = the position number of the answer.
- Ensure that the quiz is a verb, adjective, or a key phrase from the lesson.
- No proper nouns can be tested.
- Create SAMPLE SCRIPT for each topic at the end of each topic.


[POLICY]
Each quiz needs:
- A blank (test key patterns itself)
- Complete Korean translation (no blanks or tildes)


In [6]:
class PreviewQuiz(BaseModel):
    fill_in_blank_quiz : str
    korean_translation : str
    options : list[str]
    answer : str
    answer_index : int

class SampleScript(BaseModel):
    sample_script_eng : str
    sample_script_kor : str

class QuizResponse(BaseModel):
    goal : list[str]
    preview_quizzes : list[PreviewQuiz]
    sample_script : list[SampleScript]

In [7]:
response_format = type_to_response_format_param(QuizResponse)

In [8]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'$defs': {'PreviewQuiz': {'properties': {'fill_in_blank_quiz': {'title': 'Fill In Blank Quiz',
       'type': 'string'},
      'korean_translation': {'title': 'Korean Translation', 'type': 'string'},
      'options': {'items': {'type': 'string'},
       'title': 'Options',
       'type': 'array'},
      'answer': {'title': 'Answer', 'type': 'string'},
      'answer_index': {'title': 'Answer Index', 'type': 'integer'}},
     'required': ['fill_in_blank_quiz',
      'korean_translation',
      'options',
      'answer',
      'answer_index'],
     'title': 'PreviewQuiz',
     'type': 'object',
     'additionalProperties': False},
    'SampleScript': {'properties': {'sample_script_eng': {'title': 'Sample Script Eng',
       'type': 'string'},
      'sample_script_kor': {'title': 'Sample Script Kor', 'type': 'string'}},
     'required': ['sample_script_eng', 'sample_script_kor'],
     'title': 'SampleScript',
     'type': 'object',
     '

In [9]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You will provide quizzes related to the topics high school students have learned."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = QuizResponse,
    )
    return response.choices[0].message.parsed

In [29]:
response = completion(prompt)

In [11]:
response

QuizResponse(goal=['제안을 하는 표현을 활용할 수 있어요.', 'Why don’t you~?를 활용하여 제안을 할 수 있어요.'], preview_quizzes=[PreviewQuiz(fill_in_blank_quiz='Why don’t you ___ this new restaurant?', korean_translation='왜 이 새로운 레스토랑을 시도해 보지 않아?', options=['check', 'try', 'see'], answer='try', answer_index=2), PreviewQuiz(fill_in_blank_quiz='Why don’t we ___ for a walk after dinner?', korean_translation='저녁 식사 후 산책하는 건 어때?', options=['run', 'go', 'move'], answer='go', answer_index=2)], sample_script=[SampleScript(sample_script_eng="A: Why don't you try this new cafe? B: Sounds good! A: Why don't we go for a hike after lunch? B: That sounds fun!", sample_script_kor='A: 이 새로운 카페를 시도해보는 건 어때? B: 좋네! A: 점심 후에 하이킹 가는 건 어때? B: 재밌을 것 같아!')])

In [30]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent = 4)
print(response_output)

{
    "goal": [
        "의무를 표현할 때 should를 사용할 수 있어요.",
        "should~ 를 활용해서 의무를 표현할 수 있어요."
    ],
    "preview_quizzes": [
        {
            "fill_in_blank_quiz": "You ___ drink more water.",
            "korean_translation": "너는 물을 더 많이 마셔야 해.",
            "options": [
                "can",
                "should",
                "will"
            ],
            "answer": "should",
            "answer_index": 2
        },
        {
            "fill_in_blank_quiz": "You ___ apologize to her.",
            "korean_translation": "너는 그녀에게 사과해야 해.",
            "options": [
                "would",
                "should",
                "could"
            ],
            "answer": "should",
            "answer_index": 2
        }
    ],
    "sample_script": [
        {
            "sample_script_eng": "A: You should drink water frequently. B: I agree, it's very beneficial. A: You should apologize to him after the meeting. B: Absolutely, I'll do that.",
            "sample

In [18]:
print(response)

goal=['be going to ~를 활용해 계획이나 예정된 일에 대해 묻고 답할 수 있다.', 'be planning to 등의 표현을 활용해 미래 행동에 대해 이야기할 수 있다.'] preview_quizzes=[PreviewQuiz(fill_in_blank_quiz='What are you ____ do on your holiday?', korean_translation='휴일에 무엇을 할 예정이야?', options=['about to', 'going to', 'willing to'], answer='going to', answer_index=2), PreviewQuiz(fill_in_blank_quiz="I'm ____ to travel.", korean_translation='여행할 계획이야.', options=['deciding', 'planning', 'preparing'], answer='planning', answer_index=2)] sample_script=[SampleScript(sample_script_eng="A: What are you going to do this weekend? B: I'm going to visit a museum. A: What are your plans for the evening? B: I'm planning to watch a movie.", sample_script_kor='A: 이번 주말에 무엇을 할 거야? B: 박물관을 방문할 거야. A: 저녁에는 무슨 계획이 있어? B: 영화 볼 계획이야.')]


# for문 돌려서 Batch API jsonl 파일 형식 만들기

확인 사항
- A1, A2, B1, B2, C1, C2 CEFR level 별로 퀴즈만 만들어지면 되는건가?
- 여섯개의 레벨 모두 만들어야하는지, 일부 레벨(A1, B1, C1)만 만들면 되는지도 결정

1. 커리큘럼 정보 csv 파일 읽어오기
2. for문으로 한줄씩 순회하면서 프롬프트 만들어주기들어주기
3. 렌더링한 프롬프트 파일에 써주기 (처음엔 조금만, ex. 5개 커리큘럼)
4. openai batch API 요청해서 task_id 받아오기 
5. batch task 완료되면 결과 받아오고 검수하기

In [19]:
def QuizMake(data, output_filename):
    jsonl_data = []

    for i in range(len(data)):
        prompt = prompt_template.render(
            description = data.loc[i,"DESCRIPTION"],
            key_expression = data.loc[i,"Key Expression (영어)"],
            sub_expression = data.loc[i,"Sub Expression (영어)"]
        )

        quiz_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "messages" : [
                    {"role": "system", "content": "You will provide quizzes related to the topics students have learned, appropriately matching their CEFR level for middle school students."},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }

        jsonl_data.append(quiz_request)

        with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
            for item in jsonl_data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f'JSONL 파일 생성 완료 : {output_filename}-{i+1}')

## 고등학교 jsonl 파일

In [20]:
import pandas as pd

raw_data = pd.read_csv("Curriculum_High_Function.csv")
df = raw_data.loc[raw_data['구분'] == "function"]
df.head()


,textbook ID,제목,출판사,교육과정,Lesson,구분,Title,DESCRIPTION,Key Expression (영어),Key Expression (한국어),Sub Expression (영어),Unnamed: 11
0,NaN,영어,능률(김),2015,1,function,The Part You Play,be going to ~를 활용해 의도 표현하기.,I’m going to write a letter to explain my conc...,제 걱정을 분명하게 설명하기 위해 편지를 쓰려고 해요.,I plan to ~ in order to express my intentions....,NaN
1,NaN,영어,능률(김),2015,2,function,The Power of Creativity,부사 maybe를 활용해서 가능성의 정도를 표현하기,Maybe this event will occur as planned.,아마도 이 행사는 계획대로 진행될 거예요.,Perhaps ~ will work out in the expected way. I...,NaN
2,NaN,영어,능률(김),2015,3,function,Sound Life,why don’t you ~를 활용해 제안하기,Why don’t you consider taking a short break?,잠시 쉬는 것에 대해 고려해 보는 게 어때요?,Why don’t you try doing ~? Have you thought ab...,NaN
3,NaN,영어,능률(김),2015,4,function,Toward a Better World,wish 문법을 활용해 바람 표현하기.,I wish I could spend more time with my family.,가족과 더 많은 시간을 보낼 수 있으면 좋겠어요.,I hope to ~ for a better experience. I would l...,NaN
4,NaN,영어,능률(김),2015,5,function,What Matters Most,비인칭 주어 'It'을 활용해 강조 표현하기,It is crucial to maintain your health during w...,겨울철에는 건강을 유지하는 것이 중요해요.,It is essential to focus on ~ at all times. It...,NaN


In [21]:
QuizMake(df, output_filename = 'PreviewQuiz_High_batch.jsonl')

JSONL 파일 생성 완료 : PreviewQuiz_High_batch.jsonl-326


# sub_expression 해석

In [23]:
from jinja2 import Template

prompt_template = Template("""
[PURPOSE]
Please translate the content in SUB_EXPRESSION naturally into Korean.  
Make it sound casual, like talking to a friend.

[Example]
```
Input: 
    {
    "SUB_EXPRESSION": "A: What is this? B: It's a ~. A: What is that? B: It's a ~."
  }

Output:
    {
    "sub_expression_kor": "A: 이것은 뭐야? B: 이것은 ~야. A: 저것은 뭐야? B: 저것은 ~야."
  }
}
```
```
Input: 
    {
    "SUB_EXPRESSION": "A: I enjoy ~. A: I'm fond of ~."
  }

Output:
    {
    "sub_expression_kor": "A: 나는 ~을 즐겨. A: 나는 ~을 좋아해."
  }
}
```
```
Input: 
    {
    "SUB_EXPRESSION": "It’s no big deal. Don’t mention it."
  }

Output:
    {
    "sub_expression_kor": "별거 아니야. 천만에."
  }
}
```                        


[Input]
```
SUB EXPRESSION: {{sub_expression}}
```

""")

In [24]:
prompt = prompt_template.render(
    sub_expression = [
        "A: Hi, I’m (). I come from (). B: Nice to meet you. A: Hello, my name is (). I’m from (). B: Great to meet you."
    ]
)

In [25]:
print(prompt)


[PURPOSE]
Please translate the content in SUB_EXPRESSION naturally into Korean.  
Make it sound casual, like talking to a friend.

[Example]
```
Input: 
    {
    "SUB_EXPRESSION": "A: What is this? B: It's a ~. A: What is that? B: It's a ~."
  }

Output:
    {
    "sub_expression_kor": "A: 이것은 뭐야? B: 이것은 ~야. A: 저것은 뭐야? B: 저것은 ~야."
  }
}
```
```
Input: 
    {
    "SUB_EXPRESSION": "A: I enjoy ~. A: I'm fond of ~."
  }

Output:
    {
    "sub_expression_kor": "A: 나는 ~을 즐겨. A: 나는 ~을 좋아해."
  }
}
```
```
Input: 
    {
    "SUB_EXPRESSION": "It’s no big deal. Don’t mention it."
  }

Output:
    {
    "sub_expression_kor": "별거 아니야. 천만에."
  }
}
```                        


[Input]
```
SUB EXPRESSION: ['A: Hi, I’m (). I come from (). B: Nice to meet you. A: Hello, my name is (). I’m from (). B: Great to meet you.']
```



In [26]:
class Translate(BaseModel):
    sub_expression_kor : str

In [27]:
response_format = type_to_response_format_param(Translate)

In [28]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'properties': {'sub_expression_kor': {'title': 'Sub Expression Kor',
     'type': 'string'}},
   'required': ['sub_expression_kor'],
   'title': 'Translate',
   'type': 'object',
   'additionalProperties': False},
  'name': 'Translate',
  'strict': True}}

In [29]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You will provide a Korean translation."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = Translate,
    )
    return response.choices[0].message.parsed

In [30]:
response = completion(prompt)

In [31]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent = 4)
print(response_output)

{
    "sub_expression_kor": "A: 안녕, 나는 ()야. 나는 ()에서 왔어. B: 만나서 반가워. A: 안녕, 내 이름은 ()야. 나는 ()에서 왔어. B: 만나서 반가워."
}


In [32]:
print(response)

sub_expression_kor='A: 안녕, 나는 ()야. 나는 ()에서 왔어. B: 만나서 반가워. A: 안녕, 내 이름은 ()야. 나는 ()에서 왔어. B: 만나서 반가워.'


In [33]:
def Translate(data, output_filename):
    jsonl_data = []

    for i in range(len(data)):
        prompt = prompt_template.render(
            sub_expression = data.loc[i,"Sub Expression (영어)"]
        )

        quiz_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "messages" : [
                    {"role": "system", "content": "You will provide a Korean translation."},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }

        jsonl_data.append(quiz_request)

        with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
            for item in jsonl_data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f'JSONL 파일 생성 완료 : {output_filename}-{i+1}')

In [34]:
Translate(df, output_filename="Translate_High_Function_batch.jsonl")

JSONL 파일 생성 완료 : Translate_High_Function_batch.jsonl-326
